<a href="https://colab.research.google.com/github/youjinchang/SportAnalysis/blob/main/SBHAR_feature_pickle_ver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
import numpy as np
import pandas as pd
!pip install tqdm
from tqdm import tqdm
import os
import glob
import matplotlib.pyplot as plt
import random
import time
import pickle
!pip install joblib
import joblib

In [ ]:
print(pickle.format_version)

4.0


In [ ]:
import socket
hostname = socket.gethostname()
print(hostname)

5cebd66486f2


In [ ]:
!pip install tsfresh
from tsfresh import extract_features, extract_relevant_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import ComprehensiveFCParameters
from tsfresh.feature_extraction import settings
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from tensorflow.keras import models, layers
from tensorflow.keras.models import Model

     |████████████████████████████████| 97 kB 3.5 MB/s 
     |████████████████████████████████| 136 kB 7.5 MB/s 
     |████████████████████████████████| 9.8 MB 30.8 MB/s 
     |████████████████████████████████| 837 kB 26.6 MB/s 
     |████████████████████████████████| 1.1 MB 39.4 MB/s 
     |████████████████████████████████| 134 kB 37.3 MB/s 
     |████████████████████████████████| 830 kB 36.6 MB/s 
     |████████████████████████████████| 822 kB 38.2 MB/s 
     |████████████████████████████████| 802 kB 13.2 MB/s 
     |████████████████████████████████| 802 kB 38.1 MB/s 
     |████████████████████████████████| 793 kB 42.2 MB/s 
     |████████████████████████████████| 793 kB 34.8 MB/s 
     |████████████████████████████████| 791 kB 39.5 MB/s 
     |████████████████████████████████| 786 kB 19.2 MB/s 
     |████████████████████████████████| 779 kB 27.6 MB/s 
     |████████████████████████████████| 778 kB 34.6 MB/s 
     |████████████████████████████████| 776 kB 37.8 MB/s 
     |███████████

ContextualVersionConflict: ignored

In [ ]:
def MakeDir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

In [ ]:
np.random.seed(1214)

1.2 특징 추출할 X데이터 구성

In [ ]:
motionname=['WALKING',
            'WALKING_UPSTAIRS',
            'WALKING_DOWNSTAIRS',
            'SITTING',
            'STANDING',
            'LAYING']

In [ ]:
loadFromPickle = True
nsample_crop = 100
n_classes = 6

In [ ]:
# fn_pickle = './pickle/dataset_{}.pickle'.format(nsample_crop)

# if loadFromPickle:
#     if os.path.exists(fn_pickle):
#         print('{} exists....OK'.format(fn_pickle))

In [ ]:
fn_pickle = '/home/student4/work/test/dataset_100.pickle'.format(nsample_crop)

if loadFromPickle:
    if os.path.exists(fn_pickle):
        print('{} exists....OK'.format(fn_pickle))

In [ ]:
file = open('/home/student4/work/test/dataset_100.pickle'.format(nsample_crop), 'rb')

datasave = pickle.load(file)
file.close()
acc_data = datasave[0]
gyro_data = datasave[1]
y_true = datasave[2]
id_idx = datasave[3]

In [ ]:
idx_to_draw = 800
plt.figure(figsize=(4,3))
plt.plot(acc_data[idx_to_draw])
plt.title('acc_data[{}] #{}:{}'.format(idx_to_draw, y_true[idx_to_draw],motionname[y_true[idx_to_draw]]))

In [ ]:
def convert_to_one_hot(Y, C):
    Y = np.array(int(Y))
    Y = np.eye(C)[Y.reshape(-1)]
    return Y[0]

In [ ]:
acc_data = np.array(acc_data)
gyro_data = np.array(gyro_data)
y_true = np.array(y_true)
y_true_onehot = []
for i in range(len(y_true)):
    y_true_onehot.append(convert_to_one_hot(y_true[i]-1,n_classes))
y_true_onehot = np.array(y_true_onehot)

print(np.shape(acc_data))
print(np.shape(gyro_data))
print(np.shape(id_idx))
print(np.shape(y_true_onehot))

In [ ]:
data_all = np.concatenate([acc_data, gyro_data], axis=-1)
print(np.shape(data_all))

In [ ]:
vdatac = [None] * n_classes
vlabelc = [None] * n_classes
for i in range(n_classes):
    vdata = []
    vlabel = []
    indices = np.where(y_true == i+1) #y_true 가 해당 동작 번호일때의 index값을 반환
    vdata = data_all[indices,:,:]
    vlabel = y_true_onehot[indices,:]
    vdatac[i] = np.array(vdata)
    vdatac[i] = vdatac[i].reshape(vdatac[i].shape[1], vdatac[i].shape[2],vdatac[i].shape[3])
    vlabelc[i] = np.array(vlabel)
    vlabelc[i] = vlabelc[i].reshape(vlabelc[i].shape[1], vlabelc[i].shape[2])
    print('{}\t{}'.format(vdatac[i].shape, vlabelc[i].shape))

In [ ]:
len(np.arange(100)/50)

In [ ]:
idx_to_draw = 1
fig1, axarr = plt.subplots(nrows = n_classes, ncols = 2, figsize=(15, 15),dpi=300)

for i in range(n_classes):
    for j in range(2):
        axarr[i][j].plot(np.arange(100)/50,(vdatac[i][idx_to_draw])[:,j*3:j*3+3])#[j*3:j*3+3])
        axarr[i][j].set_title('Activity #{} {} Idx #{}'.format(i+1,motionname[i], idx_to_draw))

axarr[-1][0].set_xlabel('sec')
axarr[-1][1].set_xlabel('sec')
plt.tight_layout(pad=2)

#plt.savefig('./fig1.png')

In [ ]:
for i in range(n_classes):
    print(i,'\t',np.shape(vdatac[i])[0],
          '\t',round(np.shape(vdatac[i])[0]*nsample_crop*20/1000/60,1),
          'min /',round(np.shape(vdatac[i])[0]*nsample_crop*20/1000/60,1),
          'min/',round(np.shape(vdatac[i])[0]*nsample_crop*20/1000/60/60, 2),
          'hrs','\t',motionname[i])


In [ ]:
n_classes

In [ ]:
datac_all=[]
label_all=[]
num = 0
for i in range(n_classes):
    num += np.shape(vdatac[i])[0]
    num2 = np.shape(vdatac[i])[0]
    #print(num-num2, num)
    #print(np.shape(id_idx[num-num2:num]))

    measurement = vdatac[i]
    measurement = np.concatenate([measurement, id_idx[num-num2:num]], axis=-1)

    # id 열을 합쳐주는 부분
    tmp = measurement.reshape(-1, measurement.shape[-1])
    print(i, np.shape(measurement),'-->',np.shape(tmp), 'label=', np.shape(label_all), np.shape(vlabelc[i]))

    if i == 0:
        datac_all = tmp
        label_all = vlabelc[i]
    else:
        datac_all = np.vstack((datac_all, tmp))
        label_all = np.vstack((label_all, vlabelc[i]))

In [ ]:
datac_all.shape, label_all.shape

In [ ]:
 label_all[0]

In [ ]:
df = pd.DataFrame(datac_all)
df.columns = ['a_x','a_y','a_z','g_x','g_y','g_z', 'id']

In [ ]:
df

In [ ]:
df_0 = df[df['id']==0]

In [ ]:
df_0

In [ ]:
idx_to_draw = 0

ax_min = np.min(df_0['a_x'])
ax_max = np.max(df_0['a_x'])
ax_mean = np.mean(df_0['a_x'])
ax_median = np.median(df_0['a_x'])
ax_std = np.std(df_0['a_x'])
ax_skew = df_0['a_x'].skew()
#ax_dot = np.dot(df_0['a_x'], df_0['a_x'])
#ax_sum = np.sum(np.abs(np.diff(df_0['a_x'])))

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(df_0['a_x'])
plt.axhline(ax_min, color = 'red')
plt.axhline(ax_max, color = 'green')
plt.axhline(ax_mean, color = 'purple')
plt.axhline(ax_median, color = 'blue')
plt.axhline(ax_std, color = 'orange')
plt.axhline(ax_skew, color = 'black')

plt.title('acc. ax')
plt.xlabel('sample')
plt.ylabel('feature value')

In [ ]:
df.shape[0] // nsample_crop

1.2.1 Y값 만들기

In [ ]:
#y를 sereis형태로 바꾸기
Y = pd.Series(np.argmax(label_all, axis=1))

In [ ]:
Y

In [ ]:
Y.shape

*1.3* Feature extraction

In [ ]:
settings_idx = 0

In [ ]:
settings_arr = ['DecisionTree', 'RandomForest', 'SVM', 'NeuralNetwork',
                'KNN','NaiveBayes']

settings_name = settings_arr[settings_idx]
print(settings_name)

In [ ]:
if settings_idx==0: # custom 으로 선택한 feature 들 추출
    settings1 = {
            'mean': None,
            'median': None,
            'maximum': None,
            'minimum': None,
            'standard_deviation': None,
            'skewness':None,
            'kurtosis':None,
            'number_peaks': [{'n': 1}, {'n': 3}, {'n': 5}, {'n': 10}, {'n':50}],
            'abs_energy':None,
            'absolute_sum_of_changes':None,
            'mean_abs_change':None,
            'ar_coefficient': [{'coeff': 0, 'k': 10}, {'coeff': 1, 'k': 10},
                               {'coeff': 2, 'k': 10}, {'coeff': 3, 'k': 10},
                               {'coeff': 4, 'k': 10}, {'coeff': 5, 'k': 10},
                               {'coeff': 6, 'k': 10}, {'coeff': 7, 'k': 10},
                               {'coeff': 8, 'k': 10}, {'coeff': 9, 'k': 10},
                               {'coeff': 10, 'k': 10}],}

elif settings_idx==1:
    settings1 = settings.MinimalFCParameters() # 9개의 mini feature들만 추출

elif settings_idx==2:
    settings1 = settings.ComprehensiveFCParameters() # 추출가능한 모든feature 추출

elif settings_idx==3:
    settings1 = settings.EfficientFCParameters() # Comprehensive feature들 중 계산비용이 오래걸리는 feature 제외하고 추출

In [ ]:
import pickle
loadFromPickle = True

In [ ]:
out_pickle_dir = './pickle'
fn_pickle = '{}/feature_setting{}_class{}_len{}_raw.pickle'.format(out_pickle_dir,settings_idx, n_classes, nsample_crop)
print(fn_pickle)
if loadFromPickle:
    if os.path.exists(fn_pickle):
        print('{} exists....OK'.format(fn_pickle))
    else:
        print('{} does NOT exists....'.format(fn_pickle))
        loadFromPickle = False
        print('loadFromPickle....{}'.format(loadFromPickle))
        print('System will load data from RAW files')

In [ ]:
if not loadFromPickle:
    X = extract_features(df, column_id='id',
                default_fc_parameters=settings1, impute_function=impute)
    datasave = [None] * 2
    datasave[0] = X
    datasave[1] = Y
    file = open(fn_pickle, 'wb')
    pickle.dump(datasave, file)
    file.close()

else:
    print('load from pickle files')
    file = open(fn_pickle, 'rb')
    data = pickle.load(file)
    file.close()
    X = data[0]
    Y = data[1]

In [ ]:
X.shape, Y.shape

In [ ]:
X.head(3)

**1.4 Feature selection

In [ ]:
X_filtered = select_features(X, Y)

In [ ]:
X_filtered.shape

In [ ]:
from tsfresh.feature_selection.relevance import calculate_relevance_table
p_val = calculate_relevance_table(X, Y)

In [ ]:
p_val.tail(5)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.3)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

# 1.5 Classifcation

In [ ]:
classifiers = ['DecisionTree', 'RandomForest', 'SVM', 'NeuralNetwork', 'KNN','NaiveBayes']
classifiers_idx = 0
classifier_name = classifiers[classifiers_idx]
title = '{}'.format(classifier_name)

print(classifier_name)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

In [ ]:
if classifiers_idx==0:
    clf = make_pipeline(StandardScaler(), DecisionTreeClassifier(max_depth=5))

elif classifiers_idx==1:
    #RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    clf = make_pipeline(StandardScaler(), RandomForestClassifier(max_depth=5))

elif classifiers_idx==2:
    clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))

elif classifiers_idx==4:
    clf = KNeighborsClassifier(n_neighbors=10)

elif classifiers_idx==5:
    clf = GaussianNB()

elif classifiers_idx==3:
    clf = models.Sequential()
    clf.add(layers.Dense(128, activation='relu', input_shape=[X_train.shape[1]]))
    clf.add(layers.Dense(64, activation='relu'))
    clf.add(layers.Dense(32, activation='relu'))
    clf.add(layers.Dense(6, activation='sigmoid'))
    clf.summary()

    clf.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])


In [ ]:
to_Train = True

In [ ]:
if to_Train:
    print(classifier_name)
    model_outdir = './out'
    outDir = "{}/feature_{}_setting{}_class{}_len{}".
    format(model_outdir,classifier_name,settings_idx, n_classes, nsample_crop)
    MakeDir(outDir)

    if classifiers_idx != 3:
        clf.fit(X_train, y_train)
        model = "{}/{}_feature_setting{}_class{}_len{}.joblib".
        format(outDir,classifier_name,settings_idx, n_classes, nsample_crop)
        joblib.dump(clf, model)


    else:
        y_train = np.array(y_train)
        y_train_onehot = []
        for i in range(len(y_train)):
            y = convert_to_one_hot(y_train[i], 6)
            y_train_onehot.append(y)

            fnmodel = outDir+'/model_NN_class{}_len{}.h5'.format(n_classes,nsample_crop)
            #scaler = StandardScaler()
            #X_train_scaled = scaler.fit(X_train).transform(X_train)
            history = clf.fit(X_train, np.array(y_train_onehot),
                              batch_size=64 ,epochs=100) #validation_split=0.3
            clf.save(fnmodel)

else:
    model_outdir = './out'
    outDir = "{}/feature_{}_setting{}_class{}_len{}".format(model_outdir,classifier_name,settings_idx, n_classes, nsample_crop)
    print('load model files...')
    if classifiers_idx != 3:
        model = "{}/{}_feature_setting{}_class{}_len{}.joblib".format(outDir,classifier_name,settings_idx, n_classes, nsample_crop)
        print(model)
        clf = joblib.load(model)
    else:
        fnmodel = outDir+'/model_NN_class{}_len{}.h5'.format(n_classes, nsample_crop)
        clf = load_model(fnmodel)

In [ ]:
if classifiers_idx == 3:
    #X_test_scaled = scaler.fit(X_test).transform(X_test)
    expected_test = clf.predict(X_test)
    expected_train = clf.predict(X_train)
else:
    expected_test = clf.predict(X_test)
    expected_train = clf.predict(X_train)

In [ ]:
if classifiers_idx == 3:
    expected_train = np.argmax(expected_train, axis=1)
    expected_test = np.argmax(expected_test, axis=1)

In [ ]:
print(classifier_name, 'Test Result')
print(classification_report(y_test, expected_test))

In [ ]:
print(classifier_name, 'Train Result')
print(classification_report(y_train, expected_train))

In [ ]:
#Return the mean accuracy
if classifiers_idx == 3:
    y_test = np.array(y_test)
    y_test_onehot = []
    for i in range(len(y_test)):
        y = convert_to_one_hot(y_test[i], 6)
        y_test_onehot.append(y)
    score0 = clf.evaluate(X_train, np.array(y_train_onehot), verbose=1)
    score1 = clf.evaluate(X_test, np.array(y_test_onehot), verbose=1)
    score0 = score0[1]
    score1 = score1[1]

else:
    score0 = clf.score(X_train, y_train)
    score1 = clf.score(X_test, y_test)

print('Train accuracy:',score0)
print('Test accuracy:',score1)

In [ ]:
from sklearn.metrics import f1_score

ff = ['micro', 'macro', 'weighted']

f1 = f1_score(y_test, expected_test, average='weighted')
f11 = f1_score(y_train, expected_train, average='weighted')
print('Train f1-score: ',f11)
print('Test f1-score: ',f1)


In [ ]:
conf_matrix = pd.crosstab(y_test, expected_test) #행(row)이 정답, 열(col)이 맞춘거

print(conf_matrix)

In [ ]:
conf_matrix = pd.crosstab(y_train, expected_train) #행(row)이 정답, 열(col) 맞춘거

print(conf_matrix)

In [ ]:
def plot_confusion_matrix3x(actual, predicted, classes, title='ConfusionMatrix',
                            normalize=False, hide_classname = False,figsize=(4, 4),
                            dpi=36, cmap=plt.cm.viridis):

    import itertools
    from mpl_toolkits.axes_grid1 import make_axes_locatable

    if not normalize:
        conf_matrix = pd.crosstab(actual, predicted)
    else:
        conf_matrix = pd.crosstab(actual, predicted).apply(lambda r: r / r.sum(), axis=1)


    if np.shape(conf_matrix) != (len(classes), len(classes)):
        print(np.shape(conf_matrix))
        for i in range(len(classes)):
            if i not in conf_matrix.columns:
                conf_matrix[i] = 0.00
        conf_matrix = conf_matrix[[i for i in range(len(classes))]]

    fig = plt.figure(figsize=figsize, dpi=dpi)

    ax = plt.gca()
    im = ax.imshow(conf_matrix, aspect=1.0, interpolation='nearest',cmap=cmap)

    plt.title(title, size=12)

    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05, aspect=20)
    plt.colorbar(im, cax=cax)

    if hide_classname:
        classes2 = []
        for k, _ in enumerate(classes):
            classes2.append('C{}'.format(k))
        classes = classes2

    tick_marks = np.arange(len(classes))

    ax.set_xticks(tick_marks)
    ax.set_yticks(tick_marks)
    ax.set_yticklabels(classes, fontsize=9)
    ax.set_xticklabels(classes, fontsize=9, rotation=90)

    fmt = '.2f' if normalize else 'd'
    thresh = conf_matrix.max() / 2.
    thresh = 0.5

    for i, j in itertools.product(range(conf_matrix.shape[0]),
                                  range(conf_matrix.shape[1])):

        if conf_matrix[j][i] > thresh: #thresh[j]:
            color="white"

        else :
            color = "black"
        ax.text(j, i, format(conf_matrix[j][i], fmt), ha="center", va="center",color=color, fontsize=14)

        ax.set_ylabel('Actual')
        ax.set_xlabel('Predicted')
        ax.grid(False)

        return fig

In [ ]:
motionname = ['Walking', 'Upstairs', 'Downstairs', 'Sitting', 'Standing','Laying']

In [ ]:
#fig_conf = plot_confusion_matrix(testy1, pred1, class_name, title='Confusionmatrix - {}'.format(ttl1), normalize=True)

fig_conf = plot_confusion_matrix3x(y_test, expected_test, motionname, title='{}
                                   Confusion matrix'.format(classifier_name), normalize=True,
                                   figsize=(8,8), dpi=150,hide_classname = False,cmap = 'binary')


In [ ]:
ttl1fn = '{}_setting{}_class{}'.format(classifier_name, settings_idx,len(motionname))
ttl1fn = ttl1fn.replace(' ', '')

In [ ]:
fn_conf = "{}/Confusion_matrix_{}_selected_sample180.png".format(outDir,ttl1fn)#n_labeled_data

In [ ]:
print(fn_conf)
fig_conf.savefig(fn_conf)

In [ ]:
from sklearn.tree import export_graphviz
import pydotplus
from IPython.display import Image

if classifiers_idx == 0:

    feature_names = X_train.columns
    clf = DecisionTreeClassifier(max_depth=5)
    clf.fit(X_train, y_train)

    dot_data = export_graphviz(clf, out_file=None, feature_names=feature_names,class_names=motionname,
                               class_names=motionname,filled=True, rounded=True,special_characters=True)
    graph = pydotplus.graph_from_dot_data(dot_data)
    dt_img = Image(graph.create_png(), width=600, height=300)

In [ ]:
if classifiers_idx == 0:
    display(dt_img)

In [ ]:
if classifiers_idx == 3:
    from sklearn.manifold import TSNE
    emb_model = Model(clf.input, clf.layers[-2].output)
    #emb_model.summary()
    embedding = emb_model.predict(X_test)
    print(embedding.shape)

    print(y_test.shape)
    bb = y_test
    print(bb.shape)

    proj = TSNE(n_components=2).fit_transform(embedding)

In [ ]:
if classifiers_idx == 3:
    cmp = plt.get_cmap("tab20")

    plt.figure(figsize=(6,6), dpi=150)

    for i in tqdm(range(n_classes)):
        select_flag = bb == i
        plt_latent = proj[select_flag, :]
        plt.scatter(plt_latent[:, 0], plt_latent[:, 1], color=cmp(i), marker="."

    fnfig = "{}/embedding_{}.png".format(outDir, ttl1fn)#n_labeled_data
    print(fnfig)

    plt.title(ttl1fn)
    plt.gca().legend(motionname, bbox_to_anchor=(1.04,1), loc="upper left")
    plt.savefig(fnfig, bbox_inches = "tight")